In [44]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
import math
# from tensorflow.python.client import device_lib601

In [45]:
from numpy import random

In [47]:
with open('embedding_p2', 'rb') as f:
    word_embedding = pickle.load(f)

with open('vocab_p2', 'rb') as f:
    vocab = pickle.load(f)


word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# with open("processed_data/train_final", 'rb') as f:
#     full_data = pickle.load(f)

In [46]:
# f = open('vocab_p2','wb')
# pkl.dump(vocab, f, protocol=2)
# f.close()

In [47]:
# f = open('embedding_p2','wb')
# pkl.dump(word_embedding, f, protocol=2)
# f.close()

In [48]:
# f = open('fulldata_p2','wb')
# pkl.dump(full_data, f, protocol=2)
# f.close()

In [6]:
# random.shuffle(full_data)

In [7]:
# full_data[2][2]

0

In [48]:
# Parameters
mode = 'train'
num_classes = 2
batch_size = 100
vocab_size = len(vocab)
unk_vocab_size = 2
word_emb_size = len(word_embedding[0])
avg_no_sent = 10
avg_sent_size = 23
max_title_size = 22
hidden_size = 100
hidden_size2 = 50
fc_layer_hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [49]:
def word_emb_maker(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer"):
        if mode == 'train':
            unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
        Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)
        
        return Wx

In [50]:
def word_emb_maker_title(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer_top"):
        if mode == 'train':
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat_t = tf.concat([word_emb_mat, unk_word_emb_mat_t], 0)
        Wx_t = tf.nn.embedding_lookup(final_word_emb_mat_t, x)
        
        return Wx_t

In [51]:
x = tf.placeholder('int32', [None, avg_no_sent, avg_sent_size], name="x")
y = tf.placeholder('int32', [None], name="y")
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')

x_mask = tf.placeholder('int32', [None, avg_no_sent, avg_sent_size], name="x_mask")    
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 2)    

 #title
x_title = tf.placeholder('int32', [None, max_title_size], name="x_title")
x_title_mask  = tf.placeholder('bool', [None, max_title_size], name='x_title_mask')
x_title_len = tf.reduce_sum(tf.cast(x_title_mask, 'int32'), 1)

x_title_total = tf.placeholder('int32', [None, max_title_size+1], name="x_title_total")
x_title_total_mask  = tf.placeholder('bool', [None, max_title_size+1], name='x_title_total_mask')
x_title_total_len = tf.reduce_sum(tf.cast(x_title_total_mask, 'int32'), 1)

sent_mask = tf.placeholder('int32', [None, avg_no_sent], name="sent_mask")    
sent_len = tf.reduce_sum(tf.cast(sent_mask, 'int32'), 1) 

is_train = tf.placeholder('bool', [], name='is_train')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))


Wx = word_emb_maker(mode='train', x=x, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

In [52]:
Wx_title= word_emb_maker_title(mode='train',x=x_title, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

In [53]:
flat_wx = tf.reshape(Wx, [-1, avg_sent_size, word_emb_size])
flat_xlen = tf.reshape(x_len, [-1])
x_len, x_mask, flat_wx ,Wx ,flat_xlen

(<tf.Tensor 'Sum:0' shape=(?, 10) dtype=int32>,
 <tf.Tensor 'x_mask:0' shape=(?, 10, 23) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(?, 23, 100) dtype=float32>,
 <tf.Tensor 'word_embedding_layer/embedding_lookup:0' shape=(?, 10, 23, 100) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(?,) dtype=int32>)

In [54]:
with tf.variable_scope("lstm_layer"):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=input_keep_prob)
    d_cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=input_keep_prob)
    
    (fw_h, bw_h), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw, d_cell_bw, flat_wx, sequence_length=flat_xlen, dtype='float', scope='lstm')
    h = tf.concat([fw_h, bw_h], 2)

In [55]:
h

<tf.Tensor 'lstm_layer/concat:0' shape=(?, 23, 200) dtype=float32>

In [56]:
h_4d = tf.reshape(h, [-1, avg_no_sent, avg_sent_size, 2*hidden_size])

In [57]:
with tf.variable_scope("W_att_local"):
    W_att_local = tf.get_variable("W_att_local", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h = tf.reshape(h, [-1, 2*hidden_size])
    h_tanh = tf.tanh(flat_h)
    u_flat = tf.matmul(h_tanh, W_att_local)
    u_local = tf.reshape(u_flat, [-1, avg_no_sent, avg_sent_size])
    final_u = (tf.cast(x_mask, 'float') -1)*(1e7) + u_local

    c = tf.reduce_sum(tf.multiply(h_4d, tf.expand_dims(tf.nn.softmax(final_u, dim=-1), 3)), 2)

In [58]:
final_u

<tf.Tensor 'W_att_local/add:0' shape=(?, 10, 23) dtype=float32>

In [59]:
u_flat,u_local, final_u, c, sent_len

(<tf.Tensor 'W_att_local/MatMul:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'W_att_local/Reshape_1:0' shape=(?, 10, 23) dtype=float32>,
 <tf.Tensor 'W_att_local/add:0' shape=(?, 10, 23) dtype=float32>,
 <tf.Tensor 'W_att_local/Sum:0' shape=(?, 10, 200) dtype=float32>,
 <tf.Tensor 'Sum_3:0' shape=(?,) dtype=int32>)

In [60]:
with tf.variable_scope("lstm_layer_2"):
    cell_fw_2 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)
    cell_bw_2 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)

    d_cell_fw_2 = tf.contrib.rnn.DropoutWrapper(cell_fw_2, input_keep_prob=input_keep_prob)
    d_cell_bw_2 = tf.contrib.rnn.DropoutWrapper(cell_bw_2, input_keep_prob=input_keep_prob)
    
    (fw_h_2, bw_h_2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw_2, d_cell_bw_2, c, sequence_length=sent_len, dtype='float', scope='lstm2')
    h2 = tf.concat([fw_h_2, bw_h_2], 2)

In [61]:
with tf.variable_scope("W_att_local_2"):
    W_att_local_2 = tf.get_variable("W_att_local_2", shape=[2*hidden_size2, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h_2 = tf.reshape(h2, [-1, 2*hidden_size2])
    h_tanh_2 = tf.tanh(flat_h_2)
    u_flat_2 = tf.matmul(h_tanh_2, W_att_local_2)

    u_local_2 = tf.reshape(u_flat_2, [-1, avg_no_sent])
    final_u_2 = (tf.cast(sent_mask, 'float') -1)*(1e7) + u_local_2

    c2 = tf.reduce_sum(tf.multiply(h2, tf.expand_dims(tf.nn.softmax(final_u_2, dim=-1), 2)), 1)

In [62]:
final_u_2

<tf.Tensor 'W_att_local_2/add:0' shape=(?, 10) dtype=float32>

In [63]:
final_u_2.get_shape().as_list()

[None, 10]

In [64]:
u_flat_2,u_local_2, final_u_2, c2, Wx_title

(<tf.Tensor 'W_att_local_2/MatMul:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'W_att_local_2/Reshape_1:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'W_att_local_2/add:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'W_att_local_2/Sum:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'word_embedding_layer_top/embedding_lookup:0' shape=(?, 22, 100) dtype=float32>)

In [65]:
Wx_final = tf.concat([tf.expand_dims(c2,1), Wx_title],1)

In [66]:
Wx_final

<tf.Tensor 'concat:0' shape=(?, 23, 100) dtype=float32>

In [67]:
with tf.variable_scope("lstm_layer_top"):
    cell_fw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw_t = tf.contrib.rnn.DropoutWrapper(cell_fw_t, input_keep_prob=input_keep_prob)
    d_cell_bw_t = tf.contrib.rnn.DropoutWrapper(cell_bw_t, input_keep_prob=input_keep_prob)
    
    (fw_h_t, bw_h_t), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw_t, d_cell_bw_t, Wx_final, sequence_length=x_title_len, dtype='float', scope='lstm_top')
    ht = tf.concat([fw_h_t, bw_h_t], 2)

In [68]:
x_title_total_mask

<tf.Tensor 'x_title_total_mask:0' shape=(?, 23) dtype=bool>

In [69]:
with tf.variable_scope("W_att_local_top"):
    W_att_local_t = tf.get_variable("W_att_local_t", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h_t = tf.reshape(ht, [-1, 2*hidden_size])
    h_tanh_t = tf.tanh(flat_h_t)
    u_flat_t = tf.matmul(h_tanh_t, W_att_local_t)

    u_local_t = tf.reshape(u_flat_t, [-1, max_title_size+1])
    final_u_t = (tf.cast(x_title_total_mask, 'float')-1)*(1e7) + u_local_t

    ct = tf.reduce_sum(tf.multiply(ht, tf.expand_dims(tf.nn.softmax(final_u_t, dim=-1), 2)), 1)

In [70]:
final_u_t

<tf.Tensor 'W_att_local_top/add:0' shape=(?, 23) dtype=float32>

In [71]:
final_u_t.get_shape().as_list()

[None, 23]

In [72]:
with tf.variable_scope("fc_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_classes], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_classes]), name="b")
    drop_ct = tf.nn.dropout(ct, input_keep_prob)
    logits = tf.matmul(drop_ct, W) + b
    predictions = tf.argmax(logits, 1)

In [73]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [74]:
# Model Description
model_name = '3HAN_1.0'
model_dir = 'output/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    print(1)
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [75]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [36]:
# #Splitting
# data_x = [data[0:2] for data in full_data]
# data_y = [data[2] for data in full_data]
# x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, test_size=0.2, shuffle=True, stratify=data_y, random_state=0)

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, test_size=0.1, shuffle=True, stratify=y_train, random_state=0)

In [76]:
def data_prepare(x):
    #x=[news[1] for news in x_]
    num_examples = len(x)

    xx = np.zeros([num_examples, avg_no_sent, avg_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, avg_no_sent, avg_sent_size], dtype=bool)
    
    xx_title = np.zeros([num_examples, max_title_size], dtype=int)
    xx_title_mask = np.zeros([num_examples, max_title_size], dtype=bool)
    
    xx_total_title = np.zeros([num_examples, max_title_size+1], dtype=int)
    xx_total_title_mask = np.zeros([num_examples, max_title_size+1], dtype=bool)
    
    ss_mask = np.zeros([num_examples, avg_no_sent], dtype=bool)    

    for j in range(num_examples):
        for i in range(avg_no_sent):
            for k in range(avg_sent_size):
                w = x[j][1][i][k]
                if w == 'PAD':
                    xx[j][i][k] = word2id[w]
                elif w in word2id:
                    xx[j][i][k] = word2id[w]
                    xx_mask[j][i][k] = True
                else:
                    xx[j][i][k] = word2id['UNKNOWN_TOKEN']
                
    
    for j in range(num_examples):
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                xx_title[j][i] = word2id[w]
            elif w in word2id:
                xx_title[j][i] = word2id[w]
                xx_title_mask[j][i] = True
            else:
                xx_title[j][i] = word2id['UNKNOWN_TOKEN']
    
    for j in range(num_examples):
        xx_total_title_mask[j][0] = True
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                pass
            elif w in word2id:
                xx_total_title_mask[j][i+1] = True
                
    
    for j in range(num_examples):
        for i in range(avg_no_sent):
            if all(np.array(x[j][1][i])=='PAD'):
                pass
            else:
                ss_mask[j][i]=True
                
            
    return xx, xx_mask ,xx_title,xx_title_mask,xx_total_title,xx_total_title_mask,ss_mask

def eval_score(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    return f1*100, accu*100

def model(xx, yy, xx_mask, xx_title, xx_title_mask, xx_title_total_mask, ss_mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {word_emb_mat:word_embedding,
                     input_keep_prob:keep_prob,
                     is_train:train_cond,
                     x:xx[s:e], y:yy[s:e], x_mask:xx_mask[s:e],x_title:xx_title[s:e],
                     x_title_mask:xx_title_mask[s:e],
                     x_title_total_mask:xx_title_total_mask[s:e],
                     sent_mask:ss_mask[s:e]}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [107]:
def predict(xx, yy, xx_mask, xx_title, xx_title_mask, xx_title_total_mask, ss_mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {word_emb_mat:word_embedding,
                     input_keep_prob:keep_prob,
                     is_train:train_cond,
                     x:xx[s:e], y:yy[s:e], x_mask:xx_mask[s:e],x_title:xx_title[s:e],
                     x_title_mask:xx_title_mask[s:e],
                     x_title_total_mask:xx_title_total_mask[s:e],
                     sent_mask:ss_mask[s:e]}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return  y_pred
    
    return  step

In [155]:
f = open('headline.txt','r')
str1 = f.read()
f.close()

f = open('article.txt', 'r')
str2 = f.read()
f.close()

In [156]:
str1

'Donald Trump secretly worried that he may end up becoming President\n'

In [144]:
str1 = "Donald Trump secretly worried that he may end up becoming President"
str2 = "Speaking to faking News, a close aide of Mr. Trump said, Well he entered the race because he thought it would be fun. He thought he will get a stage to insult politicians and journalists, something we all want to do. And boy did he do that handsomely, every politician from Republicans to the Democrats was rubbed the wrong way by the Donald. The plan was to enjoy insulting these guys for a couple of months and then drop out but thanks to the incompetent opponents, he kept growing in popularity."

In [2]:
import pandas as pd
import jsonlines
import numpy as np
import random
import pickle as pkl
import re
import sklearn 
import pickle as pkl
import glob
from nltk import word_tokenize   
import nltk

/home/praphul/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def filter(line, whitelist):
    ##############################################################
    #   Filters out all characters which are not in whitelist    #
    ##############################################################

    return ''.join([ch for ch in line if ch in whitelist])


WHITELIST1 = "0123456789abcdefghijklmnopqrstuvwxyz'?.! "
WHITELIST2 = "0123456789abcdefghijklmnopqrstuvwxyz' "

In [17]:
def preprocess(string): 
    string = re.sub("\.\.\.", " \. ", string)
    string = re.sub(r"won't", "will not", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"n't", " not ", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"\'t", " not", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"\'m", " am", string)
    string = re.sub(r"\'d've", " would have", string)
    string = re.sub(r"\'d'y", " do you", string)
    string = re.sub(r"'", " ", string)
    return string

In [161]:
def process(str1, str2):
    str1 = filter(str1.lower(), WHITELIST2)
    str2 = filter(str2.lower(), WHITELIST1)
    str1 = preprocess(str1)
    str2 = preprocess(str2)
    str1 = str1.split()
    str2 = re.split("[\.?!]", str2)
    for k in range(len(str2)):
        str2[k] = str2[k].split()
    test = []
    temp1 = []
    temp2 = []
    for i in range(22):
        if(i<len(str1)):
            temp1.append(str1[i])
        else:
            temp1.append('PAD')
            
    pad_sent = []
    for i in range(23):
        pad_sent.append('PAD')
    for i in range(10):
        if(i<len(str2)):
            temp3 = []
            for j in range(23):
                if(j<len(str2[i])):
                    temp3.append(str2[i][j])
                else:
                    temp3.append('PAD')
            temp2.append(temp3)
        else:
            temp2.append(pad_sent)
            
    test.append(np.array([temp1, temp2]))
    return test

In [162]:
test = process(str1, str2)

In [146]:
t_test, t_mask_test ,t_title_test,t_title_mask_test,t_total_title_test,t_total_title_mask_test,tt_mask_test = data_prepare(test)


In [147]:
t_test_yy = np.zeros([len(t_test), 1])
t_test_yy = np.array(t_test_yy).reshape(-1)

In [148]:
saver.restore(sess, save_dir)

In [149]:
feed_dict = {word_emb_mat:word_embedding,
             input_keep_prob:keep_prob,
             is_train:False,
             x:t_test, y:t_test_yy, x_mask:t_mask_test,x_title:t_title_test,
             x_title_mask:t_title_mask_test,
             x_title_total_mask:t_total_title_mask_test,
             sent_mask:tt_mask_test}

In [91]:
t_test_yy

array([ 0.])

In [150]:
pred = sess.run([predictions], feed_dict)

In [151]:
pred

[array([1])]

In [45]:
# xx_train, xx_mask_train ,xx_title_train,xx_title_mask_train,xx_total_title_train,xx_total_title_mask_train,ss_mask_train = data_prepare(x_train)
# xx_val, xx_mask_val ,xx_title_val,xx_title_mask_val,xx_total_title_val,xx_total_title_mask_val,ss_mask_val = data_prepare(x_val)
# xx_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_test,xx_total_title_mask_test,ss_mask_test = data_prepare(x_test)
# yy_train = np.array(y_train).reshape(-1)
# yy_val = np.array(y_val).reshape(-1)
# yy_test = np.array(y_test).reshape(-1)

In [46]:
# xx_train.shape, xx_val.shape, xx_test.shape

((23759, 10, 23), (2640, 10, 23), (6600, 10, 23))

In [41]:
# xx_train.shape, xx_val.shape, xx_test.shape

((5000, 10, 23), (2640, 10, 23), (3000, 10, 23))

In [42]:
# import time

In [ ]:
# num_epochs = 5
# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
#     ss_mask_train = ss_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%1==0):
#         train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
#                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
#         f1_, accu_ = eval_score(y_train, train_pred)
#         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
#         val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                    xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 237 loss: 0.6621203081768776 Epoch_Time: 2567.780829668045
Saved Model Complete


In [43]:
saver.restore(sess, save_dir)

In [41]:
# val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                            xx_total_title_mask_val,ss_mask_val, train_cond=False)

# f1_, accu_ = eval_score(y_val, val_pred)
# print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)

Val: F1 Score:  80.55031910300636 Accuracy:  81.15384615384616 Loss:  0.4871215568139003


In [ ]:

# num_epochs = 5

# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
#     ss_mask_train = ss_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%1==0):
#         train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
#                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
#         f1_, accu_ = eval_score(y_train, train_pred)
#         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
#         val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                    xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 474 loss: 0.4549066013173212 Epoch_Time: 1839.4972031116486
Saved Model Complete
Train: F1 Score:  49.72728413611226 Accuracy:  53.06751054852321 Loss:  0.4262467407727543
Val: F1 Score:  82.322433748656 Accuracy:  83.46153846153847 Loss:  0.42888274788856506


In [ ]:

# num_epochs = 5

# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
#     ss_mask_train = ss_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%1==0):
# #         train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
# #                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
# #         f1_, accu_ = eval_score(y_train, train_pred)
# #         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
#         val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                    xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 711 loss: 0.4213623989231979 Epoch_Time: 1972.3159937858582
Saved Model Complete
Val: F1 Score:  81.23202327673324 Accuracy:  83.0 Loss:  0.4333428774888699


In [ ]:

# num_epochs = 5

# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
#     ss_mask_train = ss_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%1==0):
# #         train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
# #                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
# #         f1_, accu_ = eval_score(y_train, train_pred)
# #         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
#         val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                    xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 948 loss: 0.39857222561091815 Epoch_Time: 1392.0765545368195
Saved Model Complete
Val: F1 Score:  84.99530779794797 Accuracy:  85.42307692307692 Loss:  0.4068056448147847
Epoch: 2 Step: 1185 loss: 0.3807800622177526 Epoch_Time: 1123.9129800796509
Saved Model Complete
Val: F1 Score:  84.52966009755932 Accuracy:  85.96153846153847 Loss:  0.39526177713504207
Epoch: 3 Step: 1422 loss: 0.3722822886856296 Epoch_Time: 1074.4930148124695
Saved Model Complete
Val: F1 Score:  86.92716475954583 Accuracy:  87.65384615384615 Loss:  0.359285810819039
Epoch: 4 Step: 1659 loss: 0.36547351631685654 Epoch_Time: 1307.4297301769257
Saved Model Complete
Val: F1 Score:  86.30352162319552 Accuracy:  87.3076923076923 Loss:  0.35827719133633834
Epoch: 5 Step: 1896 loss: 0.35762700262451974 Epoch_Time: 1752.1305372714996
Saved Model Complete
Val: F1 Score:  87.16160584994097 Accuracy:  87.96153846153845 Loss:  0.36131645624454206
Test: F1 Score:  87.016968604988 Accuracy:  87.83333333333333 Loss:

In [43]:

# num_epochs = 5

# for i in range(num_epochs):
#     start_time = time.time()
#     random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
#     xx_train = xx_train[random]
#     yy_train = yy_train[random]
#     xx_mask_train = xx_mask_train[random]
#     xx_title_train = xx_title_train[random]
#     xx_title_mask_train = xx_title_mask_train[random]                           
#     xx_total_title_train = xx_total_title_train[random]
#     xx_total_title_mask_train = xx_total_title_mask_train[random]
#     ss_mask_train = ss_mask_train[random]
            
#     losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
#                         ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
#     print("Epoch:", i+1,"Step:", step, "loss:",losses, "Epoch_Time:",time.time() - start_time)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%1==0):
#         train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
#                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
#         f1_, accu_ = eval_score(y_train, train_pred)
#         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
#         val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                                    xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

InternalError: Dst tensor is not initialized.
	 [[Node: _arg_emb_mat_0_1/_195 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_7416__arg_emb_mat_0_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: lstm_layer_top/lstm_top/bw/bw/All/_317 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_3725_lstm_layer_top/lstm_top/bw/bw/All", tensor_type=DT_BOOL, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

In [47]:
#  train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
#                                        xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
# f1_, accu_ = eval_score(y_train, train_pred)
# print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)

# val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
#                            xx_total_title_mask_val,ss_mask_val, train_cond=False)

# f1_, accu_ = eval_score(y_val, val_pred)
# print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)

# test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_mask_test,ss_mask_test, train_cond=False)

# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)


Train: F1 Score:  87.11688604666425 Accuracy:  87.51476793248946 Loss:  0.35891008037555067
Val: F1 Score:  87.83927056343241 Accuracy:  88.1923076923077 Loss:  0.3618659835595351
Test: F1 Score:  87.2067696728499 Accuracy:  87.60606060606061 Loss:  0.35782169889320026


In [ ]:
# int(len(y_train)/batch_size) ,len(train_pred)

In [42]:
# saver.restore(sess, save_dir)